In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser
from typing import List
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
from langchain.output_parsers import OutputFixingParser
from operator import itemgetter
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI  # example LLM
from langchain.output_parsers import StructuredOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough, chain, RunnableConfig
import pandas as pd
import re
import os
from langchain.chat_models import init_chat_model
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from firecrawl import FirecrawlApp
from langchain_tavily import TavilySearch
import chromadb
from dotenv import load_dotenv
load_dotenv()
#model = "deepseek-r1:70b"
#model = "deepseek-r1:32b"
#model = "deepseek-r1:14b"
#model = "deepseek-r1:8b"
#llm = OllamaLLM(model=model)
from langchain.chat_models import init_chat_model
llm = init_chat_model("o3", model_provider="openai")
llm = init_chat_model("gpt-4.1-mini", model_provider="openai")
# llm = init_chat_model("gpt-4.1", model_provider="openai")

In [22]:
# %pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite
# %pip install -qU langchain-tavily

In [23]:
# import os
# os.getenv("LANGSMITH_TRACING")
# os.getenv("LANGSMITH_API_KEY")

In [24]:
@tool
def deep_analysis(a: str) -> str:
    """
    perform a deep analysis and then return result to a user, you will get only information that analysis is performed, that it is how ot supposed to be
    """
    print (a)
    return "analysis performed"

In [25]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search,deep_analysis]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1755190707, 'localtime': '2025-08-14 09:58'}, 'current': {'last_updated_epoch': 1755189900, 'last_updated': '2025-08-14 09:45', 'temp_c': 16.7, 'temp_f': 62.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 225, 'wind_dir': 'SW', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 80, 'cloud': 75, 'feelslike_c': 16.7, 'feelslike_f': 62.1, 'windchill_c': 13.8, 'windchill_f': 56.9, 'heatindex_c': 14.2, 'heatindex_f': 57.5, 'dewpoint_c': 13.6, 'de

In [6]:
# import getpass
# import os
# 
# if not os.environ.get("GOOGLE_API_KEY"):
#     os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
# 
# from langchain.chat_models import init_chat_model
# 
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [7]:
from langchain_deepseek import ChatDeepSeek

In [8]:
llm = ChatDeepSeek(
    model="deepseek-reasoner",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [9]:
query = "Hi!"
response = llm.invoke([{"role": "user", "content": query}])
response.text()

"Hi there! 👋 How can I help you today? Whether you have a question, need advice, or just want to chat—I'm here for it! 😊"

In [10]:
query = "what temperature in moscow now?"
response = llm.invoke([{"role": "user", "content": query}])
response.text()

'I don\'t have real-time access to current weather data. However, you can easily check the current temperature in Moscow using one of these methods:\n\n1. **Weather Websites/Apps**:  \n   Visit reliable services like:\n   - [AccuWeather](https://www.accuweather.com)\n   - [Weather.com](https://weather.com)\n   - [Yandex Weather](https://yandex.com/weather) (local source)\n\n2. **Search Engine**:  \n   Type "Moscow weather" into Google, Bing, or Yandex for an instant update.\n\n3. **Smart Devices**:  \n   Ask Alexa, Google Assistant, or Siri:  \n   *"What\'s the temperature in Moscow?"*\n\nFor reference, typical temperatures in Moscow at this time of year (May) range from **10°C to 20°C (50°F to 68°F)**, but real-time conditions may vary.'

In [26]:
model_with_tools = llm.bind_tools(tools)

In [30]:
query = "what temperature in moscow now? perform deep analysis"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'current temperature in Moscow', 'search_depth': 'basic'}, 'id': 'call_14YBUL318KjFpYK5UhtpRwsD', 'type': 'tool_call'}, {'name': 'deep_analysis', 'args': {'a': 'Analyze the current temperature conditions and trends in Moscow.'}, 'id': 'call_wbBSbreyXwLN2Ewd0I6GUMyE', 'type': 'tool_call'}]


In [28]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in San Francisco today'}, 'id': 'call_HmLn9qiBb8OaNIiYb5B6zELh', 'type': 'tool_call'}]


In [31]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [32]:
input_message = {"role": "user", "content": "what temperature in moscow now? perform deep analysis"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

Analyze the current temperature in Moscow with historical climate data, weather patterns, and forecast trends.
================================ Human Message =================================

what temperature in moscow now? perform deep analysis
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_3sOM6JhL90wLFfMZHowBuSKL)
 Call ID: call_3sOM6JhL90wLFfMZHowBuSKL
  Args:
    query: current temperature in Moscow
    search_depth: basic
  deep_analysis (call_X0ZyNPOgb0S7qA0Lp5XLMh5b)
 Call ID: call_X0ZyNPOgb0S7qA0Lp5XLMh5b
  Args:
    a: Analyze the current temperature in Moscow with historical climate data, weather patterns, and forecast trends.
================================= Tool Message =================================
Name: tavily_search

{"query": "current temperature in Moscow", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.accuweather.com/en/ru/moscow/294021/current-weat

In [17]:
input_message = {"role": "user", "content": "Search for the weather in Moscow for the period between 01.06.2025 and 01.07.2025 (dayly)"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in Moscow for the period between 01.06.2025 and 01.07.2025 (dayly)
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_b0b670a2-8a1d-45b7-8344-b425a5468e17)
 Call ID: call_0_b0b670a2-8a1d-45b7-8344-b425a5468e17
  Args:
    query: Moscow historical daily weather averages for June and July
================================= Tool Message =================================
Name: tavily_search

{"query": "Moscow historical daily weather averages for June and July", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Moscow", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Moscow', 'region': 'Moscow City', 'country': 'Russia', 'lat': 55.7522, 'lon': 37.6156, 'tz_id': 'Europe/Moscow', 'localtime_epoch': 1755116051, 'localtime': '2025-08-13 23:14'}, 'current': {'l

In [19]:
import requests

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 55.7558,
    "longitude": 37.6173,
    "start_date": "2025-07-01",
    "end_date": "2025-07-10",
    "daily": "temperature_2m_max"
}
response = requests.get(url, params=params)
data = response.json()

# # Извлечение данных по дням
# for day in data["daily"]["time"]:
#     print(f"{day}: max temp = {data['daily']['temperature_2m_max'][i]}°C")

In [20]:
data

{'latitude': 55.782074,
 'longitude': 37.576374,
 'generationtime_ms': 0.11718273162841797,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 140.0,
 'daily_units': {'time': 'iso8601', 'temperature_2m_max': '°C'},
 'daily': {'time': ['2025-07-01',
   '2025-07-02',
   '2025-07-03',
   '2025-07-04',
   '2025-07-05',
   '2025-07-06',
   '2025-07-07',
   '2025-07-08',
   '2025-07-09',
   '2025-07-10'],
  'temperature_2m_max': [15.4,
   19.7,
   25.1,
   24.7,
   20.2,
   21.9,
   27.8,
   31.8,
   31.4,
   33.5]}}